In [ ]:
"""Copy of fakenews.Net.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1r-uz1kYU8eJig3FuS49kysvzGmmLD1BF
"""

!git clone https://github.com/KaiDMML/FakeNewsNet.git

!pip install sentence_transformers


Cloning into 'FakeNewsNet'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 759 (delta 12), reused 8 (delta 3), pack-reused 735
Receiving objects: 100% (759/759), 43.99 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (156/156), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinu

In [ ]:
!gdown 1ib7nUyEoRF-Nq0k_a_2PrK_GAAwasG_5
!gdown 1a9peyeo2E6zRChOSMu3A5e78odQ2XNcL
!gdown 1oS0pFJJbg9WwVp-dhfHd0zOGfFZ4X5N9
!gdown 1JNtFhuTTQslkWCHjB4-3GNmLq33SrMTJ
!gdown 1-0j25ga98tVIbvSrm0t2Zjdzmm-PqHB6

Downloading...
From: https://drive.google.com/uc?id=1ib7nUyEoRF-Nq0k_a_2PrK_GAAwasG_5
To: /content/text_embeddings.npy
100% 71.3M/71.3M [00:01<00:00, 53.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a9peyeo2E6zRChOSMu3A5e78odQ2XNcL
To: /content/image_embeddings.npy
100% 95.0M/95.0M [00:00<00:00, 159MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oS0pFJJbg9WwVp-dhfHd0zOGfFZ4X5N9
To: /content/clip_similarities.npy
100% 186k/186k [00:00<00:00, 23.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JNtFhuTTQslkWCHjB4-3GNmLq33SrMTJ
To: /content/text_embeddings_new.npy
100% 71.3M/71.3M [00:00<00:00, 184MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-0j25ga98tVIbvSrm0t2Zjdzmm-PqHB6
From (redirected): https://drive.google.com/uc?id=1-0j25ga98tVIbvSrm0t2Zjdzmm-PqHB6&confirm=t&uuid=14b96931-cd16-4367-a4fd-71fc95234208
To: /content/image_embeddings_new.npy
100% 380M/380M [00:02<00:00, 179MB/s]


In [ ]:

"""### Sample the Data :"""

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
import requests
from io import BytesIO
from PIL import Image
from tqdm import tqdm
import concurrent.futures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Define the paths to the datasets
gossipcop_fake_path = '/content/FakeNewsNet/dataset/gossipcop_fake.csv'
gossipcop_real_path = '/content/FakeNewsNet/dataset/gossipcop_real.csv'
politifact_fake_path = '/content/FakeNewsNet/dataset/politifact_fake.csv'
politifact_real_path = '/content/FakeNewsNet/dataset/politifact_real.csv'

# Function to load data from CSV files
def load_data(file_path):
    return pd.read_csv(file_path)

# Load the data
gossipcop_fake = load_data(gossipcop_fake_path)
gossipcop_real = load_data(gossipcop_real_path)
politifact_fake = load_data(politifact_fake_path)
politifact_real = load_data(politifact_real_path)

# Add labels
gossipcop_fake['label'] = 0
politifact_fake['label'] = 0
gossipcop_real['label'] = 1
politifact_real['label'] = 1

# Combine the fake and real news datasets
fake_news = pd.concat([gossipcop_fake, politifact_fake], ignore_index=True)
real_news = pd.concat([gossipcop_real, politifact_real], ignore_index=True)
combined_news = pd.concat([fake_news, real_news], ignore_index=True)

# Shuffle the dataset
combined_news = combined_news.sample(frac=1).reset_index(drop=True)

# Combine all text columns into a single string
def combine_text_columns(row):
    return ' '.join(row.values.astype(str))

combined_news['combined_text'] = combined_news.apply(combine_text_columns, axis=1)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
"""### Extract Text and Image Embeddings :"""

#import numpy as np
from tqdm import tqdm
from huggingface_hub import login
import numpy as np
# Login to Hugging Face
# Uncomment the line below and replace 'your_huggingface_token' with your actual Hugging Face API token
# login(token="your_huggingface_token")

# Load the correct Sentence-BERT model
s_bert_model = SentenceTransformer('all-mpnet-base-v2')

# Function to encode text in batches
def batch_encode(texts, model, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch_texts)
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

# Extract text embeddings from the combined text
texts = combined_news['combined_text'].tolist()
# text_embeddings = batch_encode(texts, s_bert_model)
text_embeddings = np.load('/content/text_embeddings.npy')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# np.save('/content/drive/MyDrive/text_embeddings.npy', text_embeddings)


In [ ]:
text_embeddings.shape

(23196, 768)

In [ ]:

# Load the VGG19 model
vgg19_model = VGG19(weights='imagenet', include_top=False, pooling='avg')

# Function to preprocess and extract image embeddings
def preprocess_single_image(img_url):
    try:
        response = requests.get(img_url, timeout=5)
        img = Image.open(BytesIO(response.content)).resize((224, 224)).convert('RGB')
        img = np.array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        return vgg19_model.predict(img).flatten()
    except:
        return np.zeros((512,))

# Function to preprocess images in parallel
def preprocess_images_in_parallel(image_urls,preprocess_single_image=preprocess_single_image):
    embeddings = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(preprocess_single_image, url): url for url in image_urls}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(image_urls)):
            embeddings.append(future.result())
    return np.array(embeddings)

# Extract image embeddings in parallel for the entire dataset
image_urls = combined_news['news_url'].tolist()
# image_embeddings = preprocess_images_in_parallel(image_urls)
image_embeddings = np.load('/content/image_embeddings.npy')

# Add embeddings to the dataframe
combined_news['text_embedding'] = list(text_embeddings)
combined_news['image_embedding'] = list(image_embeddings)


80134624/80134624 [==============================] - 2s 0us/step


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# np.save('/content/drive/MyDrive/image_embeddings.npy', image_embeddings)

In [ ]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=71dedb09eedfcc18c9e50d3650299d15a4635e39d3dc535de0f39aceb6f651c9
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [ ]:

import torch
import clip
from transformers import CLIPProcessor, CLIPModel
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to compute CLIP similarity
def compute_clip_similarity(text, image_url):
    try:
        # Process text and image
        inputs = clip_processor(text=[text], images=Image.open(BytesIO(requests.get(image_url, timeout=5).content)).convert('RGB'), return_tensors="pt", padding=True)
        outputs = clip_model(**inputs)
        # Get text and image embeddings
        text_embedding = outputs.text_embeds
        image_embedding = outputs.image_embeds
        # Compute cosine similarity
        similarity = torch.nn.functional.cosine_similarity(text_embedding, image_embedding)
        return similarity.item()
    except:
        return 0.0

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
# Compute CLIP similarities
# clip_similarities = []
# for _, row in tqdm(combined_news.iterrows(), total=combined_news.shape[0]):
#     similarity = compute_clip_similarity(row['combined_text'], row['news_url'])
#     clip_similarities.append(similarity)
clip_similarities = np.load('/content/clip_similarities.npy')
combined_news['clip_similarity'] = clip_similarities

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# np.save('/content/drive/MyDrive/clip_similarities.npy', np.array(clip_similarities))

In [ ]:


# Stack embeddings for model input
text_embeddings = np.stack(combined_news['text_embedding'].values)
image_embeddings = np.stack(combined_news['image_embedding'].values)
clip_similarities = np.array(combined_news['clip_similarity']).reshape(-1, 1)

# Combine text, image embeddings, and CLIP similarity
combined_embeddings = np.concatenate([text_embeddings, image_embeddings, clip_similarities], axis=1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(combined_embeddings, combined_news['label'].values, test_size=0.2, random_state=42)


In [ ]:
"""### Train the Model :"""

# Define the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

input_shape = combined_embeddings.shape[1]
input_layer = Input(shape=(input_shape,))

# Fully connected layers
dense_layer = Dense(512, activation='relu')(input_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Define and compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Predict on validation set
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)

# Calculate validation metrics
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Validation Accuracy: {accuracy}')
print(f'Validation F1 Score: {f1}')

Epoch 1/10
580/580 [==============================] - 6s 5ms/step - loss: 0.5632 - accuracy: 0.7537 - val_loss: 0.5755 - val_accuracy: 0.7433
Epoch 2/10
580/580 [==============================] - 3s 6ms/step - loss: 0.5579 - accuracy: 0.7540 - val_loss: 0.5780 - val_accuracy: 0.7433
Epoch 3/10
580/580 [==============================] - 6s 10ms/step - loss: 0.5548 - accuracy: 0.7540 - val_loss: 0.5762 - val_accuracy: 0.7433
Epoch 4/10
580/580 [==============================] - 4s 7ms/step - loss: 0.5509 - accuracy: 0.7541 - val_loss: 0.5782 - val_accuracy: 0.7429
Epoch 5/10
580/580 [==============================] - 4s 7ms/step - loss: 0.5442 - accuracy: 0.7549 - val_loss: 0.5876 - val_accuracy: 0.7429
Epoch 6/10
580/580 [==============================] - 6s 10ms/step - loss: 0.5339 - accuracy: 0.7573 - val_loss: 0.5899 - val_accuracy: 0.7425
Epoch 7/10
580/580 [==============================] - 4s 8ms/step - loss: 0.5197 - accuracy: 0.7623 - val_loss: 0.5912 - val_accuracy: 0.7341
Epoc

In [ ]:
"""### Experiment Without CLIP Similarity:"""

# Stack embeddings for model input without CLIP similarity
combined_embeddings_no_clip = np.concatenate([text_embeddings, image_embeddings], axis=1)

# Split the data into training and validation sets
X_train_no_clip, X_val_no_clip, y_train_no_clip, y_val_no_clip = train_test_split(combined_embeddings_no_clip, combined_news['label'].values, test_size=0.2, random_state=42)

# Define the model
input_shape_no_clip = combined_embeddings_no_clip.shape[1]
input_layer_no_clip = Input(shape=(input_shape_no_clip,))

# Fully connected layers
dense_layer_no_clip = Dense(512, activation='relu')(input_layer_no_clip)
output_layer_no_clip = Dense(1, activation='sigmoid')(dense_layer_no_clip)

# Define and compile the model
model_no_clip = Model(inputs=input_layer_no_clip, outputs=output_layer_no_clip)
model_no_clip.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_no_clip.fit(X_train_no_clip, y_train_no_clip, epochs=10, batch_size=32, validation_data=(X_val_no_clip, y_val_no_clip))

# Predict on validation set
y_pred_no_clip = model_no_clip.predict(X_val_no_clip)
y_pred_no_clip = (y_pred_no_clip > 0.5).astype(int)

# Calculate validation metrics
accuracy_no_clip = accuracy_score(y_val_no_clip, y_pred_no_clip)
f1_no_clip = f1_score(y_val_no_clip, y_pred_no_clip)

print(f'Validation Accuracy without CLIP: {accuracy_no_clip}')
print(f'Validation F1 Score without CLIP: {f1_no_clip}')

Epoch 1/10
580/580 [==============================] - 3s 4ms/step - loss: 0.5636 - accuracy: 0.7533 - val_loss: 0.5739 - val_accuracy: 0.7433
Epoch 2/10
580/580 [==============================] - 3s 5ms/step - loss: 0.5575 - accuracy: 0.7540 - val_loss: 0.5748 - val_accuracy: 0.7433
Epoch 3/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5553 - accuracy: 0.7540 - val_loss: 0.5750 - val_accuracy: 0.7433
Epoch 4/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5508 - accuracy: 0.7542 - val_loss: 0.5796 - val_accuracy: 0.7431
Epoch 5/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5450 - accuracy: 0.7549 - val_loss: 0.5811 - val_accuracy: 0.7422
Epoch 6/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5335 - accuracy: 0.7584 - val_loss: 0.5859 - val_accuracy: 0.7405
Epoch 7/10
580/580 [==============================] - 3s 5ms/step - loss: 0.5186 - accuracy: 0.7631 - val_loss: 0.5981 - val_accuracy: 0.7302
Epoch 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# np.save('/content/drive/MyDrive/text_embeddings_new.npy', text_embeddings_new)

In [ ]:
## Text Model: distilbert-base-nli-mean-tokens:


# Define the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load the new text model
s_bert_model_new = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Extract new text embeddings
text_embeddings_new = np.load('/content/text_embeddings_new.npy')
# text_embeddings_new = batch_encode(texts, s_bert_model_new)

# Stack new text embeddings with existing image embeddings
combined_embeddings_new_text = np.concatenate([text_embeddings_new, image_embeddings, clip_similarities], axis=1)

# Split the data into training and validation sets
X_train_new_text, X_val_new_text, y_train_new_text, y_val_new_text = train_test_split(combined_embeddings_new_text, combined_news['label'].values, test_size=0.2, random_state=42)

# Define the model
input_shape_new_text = combined_embeddings_new_text.shape[1]
input_layer_new_text = Input(shape=(input_shape_new_text,))

# Fully connected layers
dense_layer_new_text = Dense(512, activation='relu')(input_layer_new_text)
output_layer_new_text = Dense(1, activation='sigmoid')(dense_layer_new_text)

# Define and compile the model
model_new_text = Model(inputs=input_layer_new_text, outputs=output_layer_new_text)
model_new_text.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_new_text.fit(X_train_new_text, y_train_new_text, epochs=10, batch_size=32, validation_data=(X_val_new_text, y_val_new_text))

# Predict on validation set
y_pred_new_text = model_new_text.predict(X_val_new_text)
y_pred_new_text = (y_pred_new_text > 0.5).astype(int)

# Calculate validation metrics
accuracy_new_text = accuracy_score(y_val_new_text, y_pred_new_text)
f1_new_text = f1_score(y_val_new_text, y_pred_new_text)

print(f'Validation Accuracy with new text model: {accuracy_new_text}')
print(f'Validation F1 Score with new text model: {f1_new_text}')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Epoch 1/10
580/580 [==============================] - 4s 5ms/step - loss: 0.5697 - accuracy: 0.7529 - val_loss: 0.5722 - val_accuracy: 0.7433
Epoch 2/10
580/580 [==============================] - 3s 5ms/step - loss: 0.5606 - accuracy: 0.7541 - val_loss: 0.5708 - val_accuracy: 0.7433
Epoch 3/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5602 - accuracy: 0.7543 - val_loss: 0.5783 - val_accuracy: 0.7433
Epoch 4/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5589 - accuracy: 0.7545 - val_loss: 0.5710 - val_accuracy: 0.7433
Epoch 5/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5571 - accuracy: 0.7544 - val_loss: 0.5708 - val_accuracy: 0.7431
Epoch 6/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5560 - accuracy: 0.7545 - val_loss: 0.5723 - val_accuracy: 0.7431
Epoch 7/10
580/580 [==============================] - 3s 5ms/step - loss: 0.5544 - accuracy: 0.7549 - val_loss: 0.5788 - val_accuracy: 0.7422
Epoch 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# np.save('/content/drive/MyDrive/image_embeddings_new.npy', image_embeddings_new)

In [ ]:
"""## Image Model: ResNet50:

"""

from tensorflow.keras.applications import ResNet50

# Load the new image model
resnet50_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Function to preprocess and extract image embeddings using ResNet50
def preprocess_single_image_resnet(img_url):
    try:
        response = requests.get(img_url, timeout=5)
        img = Image.open(BytesIO(response.content)).resize((224, 224)).convert('RGB')
        img = np.array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        return resnet50_model.predict(img).flatten()
    except:
        return np.zeros((2048,))  # Adjusted for ResNet50 output

# Extract new image embeddings in parallel using ResNet50
image_embeddings_new = np.load('/content/image_embeddings_new.npy')
# image_embeddings_new = preprocess_images_in_parallel(image_urls, preprocess_single_image_resnet)

# Stack existing text embeddings with new image embeddings
combined_embeddings_new_image = np.concatenate([text_embeddings, image_embeddings_new, clip_similarities], axis=1)

# Split the data into training and validation sets
X_train_new_image, X_val_new_image, y_train_new_image, y_val_new_image = train_test_split(combined_embeddings_new_image, combined_news['label'].values, test_size=0.2, random_state=42)

# Define the model
input_shape_new_image = combined_embeddings_new_image.shape[1]
input_layer_new_image = Input(shape=(input_shape_new_image,))

# Fully connected layers
dense_layer_new_image = Dense(512, activation='relu')(input_layer_new_image)
output_layer_new_image = Dense(1, activation='sigmoid')(dense_layer_new_image)

# Define and compile the model
model_new_image = Model(inputs=input_layer_new_image, outputs=output_layer_new_image)
model_new_image.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_new_image.fit(X_train_new_image, y_train_new_image, epochs=10, batch_size=32, validation_data=(X_val_new_image, y_val_new_image))

# Predict on validation set
y_pred_new_image = model_new_image.predict(X_val_new_image)
y_pred_new_image = (y_pred_new_image > 0.5).astype(int)

# Calculate validation metrics
accuracy_new_image = accuracy_score(y_val_new_image, y_pred_new_image)
f1_new_image = f1_score(y_val_new_image, y_pred_new_image)

print(f'Validation Accuracy with new image model: {accuracy_new_image}')
print(f'Validation F1 Score with new image model: {f1_new_image}')

94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/10
580/580 [==============================] - 4s 6ms/step - loss: 0.5632 - accuracy: 0.7528 - val_loss: 0.5732 - val_accuracy: 0.7433
Epoch 2/10
580/580 [==============================] - 4s 6ms/step - loss: 0.5583 - accuracy: 0.7540 - val_loss: 0.5733 - val_accuracy: 0.7433
Epoch 3/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5557 - accuracy: 0.7540 - val_loss: 0.5765 - val_accuracy: 0.7433
Epoch 4/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5526 - accuracy: 0.7540 - val_loss: 0.5769 - val_accuracy: 0.7431
Epoch 5/10
580/580 [==============================] - 2s 4ms/step - loss: 0.5476 - accuracy: 0.7545 - val_loss: 0.5785 - val_accuracy: 0.7429
Epoch 6/10
580/580 [==============================] - 3s 5ms/step - loss: 0.5395 - accuracy: 0.7561 - val_loss: 0.5842 - val_accuracy: 0.7412
Epoch 7/10
580/580 [==============================] - 3s 5ms/step - loss: 0.5290 - 

In [ ]:


"""### New ones :"""

# Define the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

def run_experiment(with_clip=True, num_runs=5):
    accuracy_list = []
    f1_list = []

    for _ in range(num_runs):
        # Split the data into training and validation sets
        if with_clip:
            combined_embeddings = np.concatenate([text_embeddings, image_embeddings, clip_similarities], axis=1)
        else:
            combined_embeddings = np.concatenate([text_embeddings, image_embeddings], axis=1)

        X_train, X_val, y_train, y_val = train_test_split(combined_embeddings, combined_news['label'].values, test_size=0.2, random_state=42)

        # Define the model
        input_shape = combined_embeddings.shape[1]
        input_layer = Input(shape=(input_shape,))

        # Fully connected layers
        dense_layer = Dense(512, activation='relu')(input_layer)
        output_layer = Dense(1, activation='sigmoid')(dense_layer)

        # Define and compile the model
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

        # Predict on validation set
        y_pred = model.predict(X_val)
        y_pred = (y_pred > 0.5).astype(int)

        # Calculate validation metrics
        accuracy = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)

        accuracy_list.append(accuracy)
        f1_list.append(f1)

    return np.mean(accuracy_list), np.std(accuracy_list), np.mean(f1_list), np.std(f1_list)

# Run experiments
accuracy_with_clip, std_accuracy_with_clip, f1_with_clip, std_f1_with_clip = run_experiment(with_clip=True)
accuracy_without_clip, std_accuracy_without_clip, f1_without_clip, std_f1_without_clip = run_experiment(with_clip=False)

print(f'With CLIP - Accuracy: {accuracy_with_clip} ± {std_accuracy_with_clip}, F1: {f1_with_clip} ± {std_f1_with_clip}')
print(f'Without CLIP - Accuracy: {accuracy_without_clip} ± {std_accuracy_without_clip}, F1: {f1_without_clip} ± {std_f1_without_clip}')


145/145 [==============================] - 0s 2ms/step
With CLIP - Accuracy: 0.6679741379310344 ± 0.009064645949131984, F1: 0.7914854520791729 ± 0.008085601139051764
Without CLIP - Accuracy: 0.6879741379310345 ± 0.013596661477339587, F1: 0.8079118521359083 ± 0.01163774603937375
